# RFM 分析

使用 UCI Online Retail 資料集

* [https://archive.ics.uci.edu/ml/datasets/Online+Retail](https://archive.ics.uci.edu/ml/datasets/Online+Retail)

參考資料：

* https://en.wikipedia.org/wiki/RFM_(market_research)

## 下載資料集

In [ ]:
# 在colab中可以用這個
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

In [ ]:
# 在windows上，用這個試試看
!curl -o "Online Retail.xlsx" https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

## 讀取資料集

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Retail_df = pd.read_excel("Online Retail.xlsx")

In [ ]:
Retail_df['amount'] = Retail_df['Quantity']*Retail_df['UnitPrice']

In [ ]:
Retail_df.head(1)

In [ ]:
Retail_df.info()

In [ ]:
Retail_df['InvoiceDate'].min()

In [ ]:
Retail_df['InvoiceDate'].max()

## 刪除缺失值

In [ ]:
df = Retail_df[Retail_df['CustomerID'].notnull()]

In [ ]:
ref_date = df['InvoiceDate'].max()

## 計算 R, F, M 值

In [ ]:
# 計算最近一次購買時間
recency = df['InvoiceDate'].groupby(df['CustomerID']).max()
# 計算購買的次數
frequency = df['InvoiceDate'].groupby(df['CustomerID']).count()
# 計算購買的金額
monetary = df['amount'].groupby(df['CustomerID']).sum()

In [ ]:
# 計算上一次購買間隔天數(幾天前)
r_interval = (ref_date - recency).dt.days

In [ ]:
# 計算 R, F, M 值
r_score = pd.cut(r_interval, 5, labels=[5, 4, 3, 2, 1])
f_score = pd.cut(frequency, 5, labels=[1, 2, 3, 4, 5])
m_score = pd.cut(monetary, 5, labels=[1, 2, 3, 4, 5])

In [ ]:
# 合併Series成DataFrame: 方法一
rfm = {'r_score':r_score, 'f_score':f_score, 'm_score':m_score}
rfm_df = pd.DataFrame(rfm)

In [ ]:
# 合併Series成DataFrame: 方法二
rfm_df = pd.concat([r_score, f_score, m_score], axis=1)
rfm_df.columns = ['r_score', 'f_score', 'm_score']

In [ ]:
rfm_df.info()

In [ ]:
rfm_df = rfm_df.astype(int)

In [ ]:
rfm_df.head()

In [ ]:
rfm_df.info()

## 計算 RFM 總分

兩種方式：

* 直接評估 R, F, M 的重要性，再做加權平均成為一個數字。
* 直接串接 R, F, M 各自的值，秀出一個字串。

In [ ]:
# 方法一：加權平均
rfm_df['rfm_score'] = rfm_df['r_score'] * 0.5 + rfm_df['f_score'] * 0.3 + rfm_df['m_score'] * 0.2

In [ ]:
# 方法二：RFM组合
rfm_df_tmp = rfm_df.copy()
rfm_df_tmp['r_score'] = rfm_df_tmp['r_score'].astype('str')
rfm_df_tmp['f_score'] = rfm_df_tmp['f_score'].astype('str')
rfm_df_tmp['m_score'] = rfm_df_tmp['m_score'].astype('str')
rfm_df['rfm_comb'] = rfm_df_tmp['r_score']+rfm_df_tmp['f_score']+rfm_df_tmp['m_score']

In [ ]:
rfm_df.head()

In [ ]:
rfm_df.sort_values('rfm_score', ascending=False)